In [5]:

import os
import re
#import hunspell
from termcolor import colored
import numpy as np
import PyPDF2
import pandas as pd
from pylatexenc.latexencode import unicode_to_latex
from chardet.universaldetector import UniversalDetector
import textwrap as tr

In [11]:
def return_header(text):

  indx = text.index("\\begin{document}\n")
  text = text[:indx]
  
  t = []
  for l in text:
      if l.startswith("%"):
          continue
      t.append(l)
  
  a = []
  for l in t:
      if l == '\n':
          continue
      a.append(l)
  
  b = []
  for l in a:
      if l.startswith('\\documentclass'):
          continue
      b.append(l)
  
  c = []
  for l in b:
      if l.startswith('\\usepackage'):
          continue
      c.append(l)
  
  d = ''
  for i in c:
      d += i
  return(d)

In [12]:
def return_metadata(text,keys):
  metadata = {}

  while True:
    try:
       i = text.index('\\')
       j = text.index('{', i)
       key = text[i + 1: j]
       k = j + 1
       abierto = 1

       while abierto > 0 and k < len(text):
         if text[k] == '{':
           abierto += 1
         if text[k] == '}':
           abierto -= 1
         k += 1

       if key in keys:
         metadata[key] = text[j + 1: k - 1].strip()

       text = text[k:]
    except ValueError:
       break
    except IndexError:
       break

  for required_key in keys:
    if required_key not in metadata:
      metadata[required_key] = None

  return(metadata)


In [27]:
def process(l):

  detector = UniversalDetector()                         
  detector.reset()
  for line in open(l+".tex","rb"):                      
    detector.feed(line)
    if detector.done: break
  detector.close()
  encoding = detector.result['encoding']

  f = open(l+'.tex','r',encoding=encoding)
  text = f.readlines()
  f.close()

  keys = ['contriblanguage','contribtype','thematicarea','title','author','abstract','subtitle','keywords','institute']

  d = return_header(text)
  m = return_metadata(d,keys)

#  m['title']  = clean_title(m['title'],m['subtitle'])
#  m['author']  = clean_author(m['author'])
#  m['keywords']  = clean_keywords(m['keywords'],keywordList)
#  m['abstract'] = clean_abstract(m['abstract'])
#  m['contriblanguage']  = language[m['contriblanguage']]
#  m['contribtype'] = contribtype[m['contribtype']]
#  m['thematicarea'] = thematicarea[m['thematicarea']]
#  m['num_authors'] = len(m['author'])



#  f = open(l+'.pdf', 'rb')
#  fileReader = PyPDF2.PdfReader(f)
#  m['npages'] = len(fileReader.pages)


  return(m)


In [28]:
m=process('928')

In [29]:
m.keys()

dict_keys(['contriblanguage', 'contribtype', 'thematicarea', 'title', 'author', 'institute', 'abstract', 'keywords', 'subtitle'])

In [30]:
m['institute']

"Facultad de Ciencias Astron\\'omicas y Geof\\'isicas, UNLP, Argentina \\and\n  Instituto de Astrof\\'isica de La Plata, CONICET--UNLP, Argentina \\and\n  Consejo Nacional de Investigaciones Cient\\'ificas y T\\'ecnicas, Argentina \\and\n  Waterloo Centre for Astrophysics, University of Waterloo, Canada \\and\n  Perimeter Institute for Theoretical Physics, Canada \\and\n  Instituto de F\\'isica, Universidade Federal do Rio Grande do Sul, Brasil \\and\n  University of Hertfordshire, Reino Unido"

In [41]:
# Expresión regular para capturar solo los números dentro de \inst{}
pattern = r"\\inst\{([0-9,]+)\}"

# Extraer las coincidencias
matches = re.findall(pattern, m['author'])

# Crear la lista de listas de números (convertidos a enteros)
inst_list = [[int(num) for num in inst.split(',')] for inst in matches]

# Mostrar el resultado
print(inst_list)


# Diccionario para reemplazar los comandos LaTeX más comunes
latex_replacements = {
    r"\'a": "á",
    r"\'e": "é",
    r"\'i": "í",
    r"\'o": "ó",
    r"\'u": "ú",
    r"\'A": "Á",
    r"\'E": "É",
    r"\'I": "Í",
    r"\'O": "Ó",
    r"\'U": "Ú",
    r"\'n": "ñ",
    r"\'N": "Ñ",
    r"\\&": "&",
    r"---": "—",
    r"--": "–",
}

# Eliminar el comando \institute{}
institute_string_clean = re.sub(r"\\institute\{|\}", "", m['institute'])

# Reemplazar los comandos de LaTeX por texto plano
for latex, plain_text in latex_replacements.items():
    institute_string_clean = institute_string_clean.replace(latex, plain_text)

# Expresión regular para capturar cada afiliación separada por \and
pattern = r"(.*?)(?:\\and|$)"

# Extraer las coincidencias
affiliations = re.findall(pattern, institute_string_clean, re.DOTALL)

# Limpiar espacios en blanco y caracteres extra
affiliations = [affil.strip() for affil in affiliations]

# Mostrar el resultado
print(affiliations)

# Crear una lista de listas con las afiliaciones de cada autor
author_affiliations = []

# Recorremos cada lista de instancias (instancias de afiliaciones)
for inst in inst_list:
    # Para cada lista de instancias de un autor, obtenemos la afiliación correspondiente
    # Restamos 1 al índice porque las instancias están numeradas desde 1, no desde 0
    author_affiliations.append([affiliations[i-1] for i in inst])

# Mostrar el resultado
for i, affil in enumerate(author_affiliations, 1):
    print(f"Autor {i}: {affil}")

# Crear el string de salida
output = []

for i, insts in enumerate(inst_list):
    prefix1 = chr(65 + i)  # 'A' es 65 en ASCII
    # Convertir el índice a un número entero para acceder a la afiliación
    affiliation_index = int(insts[0]) - 1  # Restar 1 para el índice cero
    output.append(f"A{prefix1}({affiliations[affiliation_index]}) ")

"".join(output)

[[1, 2, 3], [4, 5], [6], [1, 2, 3], [7], [6], [6], [6], [6], [6], [6]]
['Facultad de Ciencias Astronómicas y Geofísicas, UNLP, Argentina', 'Instituto de Astrofísica de La Plata, CONICET–UNLP, Argentina', 'Consejo Nacional de Investigaciones Científicas y Técnicas, Argentina', 'Waterloo Centre for Astrophysics, University of Waterloo, Canada', 'Perimeter Institute for Theoretical Physics, Canada', 'Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil', 'University of Hertfordshire, Reino Unido', '']
Autor 1: ['Facultad de Ciencias Astronómicas y Geofísicas, UNLP, Argentina', 'Instituto de Astrofísica de La Plata, CONICET–UNLP, Argentina', 'Consejo Nacional de Investigaciones Científicas y Técnicas, Argentina']
Autor 2: ['Waterloo Centre for Astrophysics, University of Waterloo, Canada', 'Perimeter Institute for Theoretical Physics, Canada']
Autor 3: ['Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil']
Autor 4: ['Facultad de Ciencias Astronómicas y

In [49]:
# Crear el string de salida
output = []

for i, insts in enumerate(inst_list):
    prefix1 = chr(65 + i)  # 'A' es 65 en ASCII
    # Convertir el índice a un número entero para acceder a la afiliación
    affiliation_index = int(insts[0]) - 1  # Restar 1 para el índice cero
    output.append(f"A{prefix1}({affiliations[affiliation_index]}) ")

In [50]:
"".join(output)

'AA(Facultad de Ciencias Astronómicas y Geofísicas, UNLP, Argentina) AB(Waterloo Centre for Astrophysics, University of Waterloo, Canada) AC(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) AD(Facultad de Ciencias Astronómicas y Geofísicas, UNLP, Argentina) AE(University of Hertfordshire, Reino Unido) AF(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) AG(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) AH(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) AI(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) AJ(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) AK(Instituto de Física, Universidade Federal do Rio Grande do Sul, Brasil) '

In [34]:
[[r for r in affiliations[h]] for h in inst_list]

TypeError: list indices must be integers or slices, not list

In [2]:

def extract_affiliations_from_tex(tex_file):
    # Leer el archivo .tex
    with open(tex_file, 'r', encoding='utf-8') as file:
        tex_content = file.read()

    # Expresión regular para extraer autores y sus instancias
    author_pattern = re.compile(r'\\author\s*\{(.*?)\}', re.DOTALL)
    authors_block = author_pattern.search(tex_content).group(1)

    # Extraer autores y su(s) número(s) de instancias
    authors_with_inst = re.findall(r'([A-Za-z\.\s\'\\\&]+)\s*\\inst\{(\d+(?:,\d+)*)\}', authors_block)

    # Expresión regular para extraer las instituciones
    institute_pattern = re.compile(r'\\institute\s*\{(.*?)\}', re.DOTALL)
    institutes_block = institute_pattern.search(tex_content).group(1)

    # Extraer afiliaciones divididas por \and
    affiliations = re.split(r'\s*\\and\s*', institutes_block)

    # Crear un diccionario de autores con sus afiliaciones
    author_affiliations = {}
    for author, inst_numbers in authors_with_inst:
        # Obtener las afiliaciones correspondientes a los números de instancias
        author_affiliations[author.strip()] = [affiliations[int(i)-1].strip() for i in inst_numbers.split(',')]

    return author_affiliations




In [3]:
# Ejemplo de uso:
affiliations = extract_affiliations_from_tex('928.tex')
for author, affiliation in affiliations.items():
    print(f"{author}: {', '.join(affiliation)}")

In [4]:
affiliations

{}

In [81]:
import re

# Contenido del archivo .bbl
bbl_content = r"""
\begin{thebibliography}{56}
\providecommand{\natexlab}[1]{#1}

\bibitem[{{Abdollahi} et~al.(2022)}]{abdollahi2022}
{Abdollahi} S., et~al., 2022, \apjs, 260, 53

\bibitem[{{Ballet} et~al.(2023)}]{ballet2023}
{Ballet} J., et~al., 2023, arXiv e-prints, arXiv:2307.12546

\bibitem[{{Benaglia}(2010)}]{jaenproc2010}
{Benaglia} P., 2010, J.~{Mart{\'\i}}, P.L. {Luque-Escamilla}, J.A. {Combi}
  (Eds.), \textit{High Energy Phenomena in Massive Stars}, \textit{Astronomical
  Society of the Pacific Conference Series}, vol. 422, 111

\bibitem[{{Benaglia}(2016)}]{benaglia2016}
{Benaglia} P., 2016, \pasa, 33, e017

\bibitem[{{Benaglia} et~al.(2001{\natexlab{a}}){Benaglia}, {Cappa} \&
  {Koribalski}}]{benagliamdot200}
{Benaglia} P., {Cappa} C.E., {Koribalski} B.S., 2001{\natexlab{a}}, \aap, 372,
  952

\bibitem[{{Benaglia} et~al.(2006){Benaglia}, {Koribalski} \& {Albacete
  Colombo}}]{benaglia2006}
{Benaglia} P., {Koribalski} B., {Albacete Colombo} J.F., 2006, \pasa, 23, 50
\end{thebibliography}{56}
"""

# Expresión regular para capturar los elementos entre \bibitem y el final
pattern = r"(.*?)(?:\\bibitem|\\end\{thebibliography\}$)"

# Buscar todas las coincidencias
bibliography_items = re.findall(pattern, bbl_content, re.DOTALL)

# Limpiar los elementos
bibliography_items = [re.sub(r"\n","", item) for item in bibliography_items]


bibliography_items = [re.sub(r"^\[.*?\]", "", item).strip() for item in bibliography_items]
bibliography_items = [re.sub(r"^\{.*?\}", "", item).strip() for item in bibliography_items]

# Mostrar la lista resultante
for index, item in enumerate(bibliography_items[1:]):
    print(f"Elemento {index + 1}: {item}\n")


Elemento 1: {Abdollahi} S., et~al., 2022, \apjs, 260, 53

Elemento 2: {Ballet} J., et~al., 2023, arXiv e-prints, arXiv:2307.12546

Elemento 3: {Benaglia} P., 2010, J.~{Mart{\'\i}}, P.L. {Luque-Escamilla}, J.A. {Combi}  (Eds.), \textit{High Energy Phenomena in Massive Stars}, \textit{Astronomical  Society of the Pacific Conference Series}, vol. 422, 111

Elemento 4: {Benaglia} P., 2016, \pasa, 33, e017

Elemento 5: {Benaglia} P., {Cappa} C.E., {Koribalski} B.S., 2001{\natexlab{a}}, \aap, 372,  952



In [63]:
import re

# Contenido del archivo .bbl
bbl_content = r"""
\begin{thebibliography}{56}
\providecommand{\natexlab}[1]{#1}

\bibitem[{{Abdollahi} et~al.(2022)}]{abdollahi2022}
{Abdollahi} S., et~al., 2022, \apjs, 260, 53

\bibitem[{{Ballet} et~al.(2023)}]{ballet2023}
{Ballet} J., et~al., 2023, arXiv e-prints, arXiv:2307.12546

\bibitem[{{Benaglia}(2010)}]{jaenproc2010}
{Benaglia} P., 2010, J.~{Mart{\'\i}}, P.L. {Luque-Escamilla}, J.A. {Combi}
  (Eds.), \textit{High Energy Phenomena in Massive Stars}, \textit{Astronomical
  Society of the Pacific Conference Series}, vol. 422, 111

\bibitem[{{Benaglia}(2016)}]{benaglia2016}
{Benaglia} P., 2016, \pasa, 33, e017

\bibitem[{{Benaglia} et~al.(2001{\natexlab{a}}){Benaglia}, {Cappa} \&
  {Koribalski}}]{benagliamdot200}
{Benaglia} P., {Cappa} C.E., {Koribalski} B.S., 2001{\natexlab{a}}, \aap, 372,
  952

\bibitem[{{Benaglia} et~al.(2006){Benaglia}, {Koribalski} \& {Albacete
  Colombo}}]{benaglia2006}
{Benaglia} P., {Koribalski} B., {Albacete Colombo} J.F., 2006, \pasa, 23, 50
\end{thebibliography}{56}
"""

# Expresión regular para capturar los elementos entre \bibitem y el final
pattern = pattern = r"(.*?)(?:\\bibitem|$)"

# Buscar todas las coincidencias
bibliography_items = re.findall(pattern, bbl_content, re.DOTALL)
print(bibliography_items)

# Limpiar los elementos y eliminar el \bibitem[...]{...}
bibliography_items = [re.sub(r"\\bibitem\{.*?\} ", "", item.strip()) for item in bibliography_items]

# Mostrar la lista resultante
for index, item in enumerate(bibliography_items):
    print(f"Elemento {index + 1}: {item}\n")


['\n\\begin{thebibliography}{56}\n\\providecommand{\\natexlab}[1]{#1}\n\n', '[{{Abdollahi} et~al.(2022)}]{abdollahi2022}\n{Abdollahi} S., et~al., 2022, \\apjs, 260, 53\n\n', '[{{Ballet} et~al.(2023)}]{ballet2023}\n{Ballet} J., et~al., 2023, arXiv e-prints, arXiv:2307.12546\n\n', "[{{Benaglia}(2010)}]{jaenproc2010}\n{Benaglia} P., 2010, J.~{Mart{\\'\\i}}, P.L. {Luque-Escamilla}, J.A. {Combi}\n  (Eds.), \\textit{High Energy Phenomena in Massive Stars}, \\textit{Astronomical\n  Society of the Pacific Conference Series}, vol. 422, 111\n\n", '[{{Benaglia}(2016)}]{benaglia2016}\n{Benaglia} P., 2016, \\pasa, 33, e017\n\n', '[{{Benaglia} et~al.(2001{\\natexlab{a}}){Benaglia}, {Cappa} \\&\n  {Koribalski}}]{benagliamdot200}\n{Benaglia} P., {Cappa} C.E., {Koribalski} B.S., 2001{\\natexlab{a}}, \\aap, 372,\n  952\n\n', '[{{Benaglia} et~al.(2006){Benaglia}, {Koribalski} \\& {Albacete\n  Colombo}}]{benaglia2006}\n{Benaglia} P., {Koribalski} B., {Albacete Colombo} J.F., 2006, \\pasa, 23, 50\n\\end{th